In [1]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyvista
import viskex

from dolfinx import fem, mesh, io, plot
from mpi4py import MPI

import commons, utils

In [2]:
os.environ['VISKEX_PYVISTA_BACKEND'] = "trame"
pyvista.global_theme.transparent_background = True
pyvista.OFF_SCREEN = True

In [3]:
markers = commons.Markers()

In [4]:
comm = MPI.COMM_WORLD
domain, ct, ft = io.gmshio.read_from_msh("output/reaction_distribution/40-40-75/unrefined/1.0/mesh.msh", comm)

Info    : Reading 'output/reaction_distribution/40-40-75/unrefined/1.0/mesh.msh'...
Info    : 22 entities
Info    : 76418 nodes
Info    : 470810 elements
Info    : Done reading 'output/reaction_distribution/40-40-75/unrefined/1.0/mesh.msh'


In [20]:
pv_plotter = viskex.DolfinxPlotter.plot_mesh_tags(domain, ct, "facets", show_edges=False, opacity=1)
plotter = pyvista.Plotter()
clipped = plotter.add_mesh_clip_plane(pv_plotter.mesh, normal=[1, 0, 0], normal_rotation=0, implicit=False)
plotter.remove_scalar_bar()
plotter.hide_axes()
plotter.view_zy()
plotter.add_point_labels(points=np.array([[0, -15e-6, 2.5e-6],[0, 0, 30e-6]]), labels=["solid electrolyte", "positive active material"], text_color="white", show_points=False, shadow=False, fill_shape=False, render=False, shape_opacity=0)
plotter.hide_axes_all()
_ = plotter.screenshot("figures/secondary_current/cell_mesh.png")

In [6]:
positive_am_domain, entity_map, vertex_map, geom_map = mesh.create_submesh(domain, domain.topology.dim, ct.indices[(ct.values == markers.positive_am)])
tdim = domain.topology.dim
fdim = tdim - 1
c_to_f = domain.topology.connectivity(tdim, fdim)
f_map = domain.topology.index_map(fdim)
all_facets = f_map.size_local + f_map.num_ghosts
all_values = np.zeros(all_facets, dtype=np.int32)
all_values[ft.indices] = ft.values

positive_am_domain.topology.create_entities(fdim)
subf_map = positive_am_domain.topology.index_map(fdim)
positive_am_domain.topology.create_connectivity(tdim, fdim)
c_to_f_sub = positive_am_domain.topology.connectivity(tdim, fdim)
num_sub_facets = subf_map.size_local + subf_map.num_ghosts
sub_values = np.empty(num_sub_facets, dtype=np.int32)
for i, entity in enumerate(entity_map):
    parent_facets = c_to_f.links(entity)
    child_facets = c_to_f_sub.links(i)
    for child, parent in zip(child_facets, parent_facets):
        sub_values[child] = all_values[parent]
positive_am_ft = mesh.meshtags(positive_am_domain, positive_am_domain.topology.dim - 1, np.arange(
    num_sub_facets, dtype=np.int32), sub_values)
positive_am_domain.topology.create_connectivity(positive_am_domain.topology.dim - 1, positive_am_domain.topology.dim)

In [7]:
pv_plot = viskex.DolfinxPlotter.plot_mesh_tags(positive_am_domain, positive_am_ft, "facets", show_edges=False, opacity=1)
pv_plot.remove_scalar_bar()
pv_plot.hide_axes()
_ = pv_plot.screenshot("figures/secondary_current/positive_am.png")

In [8]:
electrolyte_domain, entity_map, vertex_map, geom_map = mesh.create_submesh(domain, domain.topology.dim, ct.indices[(ct.values == markers.electrolyte)])
tdim = domain.topology.dim
fdim = tdim - 1
c_to_f = domain.topology.connectivity(tdim, fdim)
f_map = domain.topology.index_map(fdim)
all_facets = f_map.size_local + f_map.num_ghosts
all_values = np.zeros(all_facets, dtype=np.int32)
all_values[ft.indices] = ft.values

electrolyte_domain.topology.create_entities(fdim)
subf_map = electrolyte_domain.topology.index_map(fdim)
electrolyte_domain.topology.create_connectivity(tdim, fdim)
c_to_f_sub = electrolyte_domain.topology.connectivity(tdim, fdim)
num_sub_facets = subf_map.size_local + subf_map.num_ghosts
sub_values = np.empty(num_sub_facets, dtype=np.int32)
for i, entity in enumerate(entity_map):
    parent_facets = c_to_f.links(entity)
    child_facets = c_to_f_sub.links(i)
    for child, parent in zip(child_facets, parent_facets):
        sub_values[child] = all_values[parent]
electrolyte_ft = mesh.meshtags(electrolyte_domain, electrolyte_domain.topology.dim - 1, np.arange(
    num_sub_facets, dtype=np.int32), sub_values)
electrolyte_domain.topology.create_connectivity(electrolyte_domain.topology.dim - 1, electrolyte_domain.topology.dim)

In [23]:
pv_plotter_elec = viskex.DolfinxPlotter.plot_mesh_tags(electrolyte_domain, electrolyte_ft, "facets", show_edges=False, opacity=1)
pv_plotter_elec.remove_scalar_bar()
pv_plotter_elec.hide_axes()
_ = pv_plotter_elec.screenshot("figures/secondary_current/electrolyte.png")

In [10]:
def get_simulation_metafile_path(Wa_p, kr, gamma, resolution, kinetics="butler_volmer"):
    return os.path.join("output/reaction_distribution/40-40-75", "unrefined", str(resolution), kinetics, f"0.001-{Wa_p}-{kr}", str(gamma), "simulation.json")

In [11]:
def read_simulation_metafile(metafile_path):
    with open(metafile_path, "r") as fp:
        return json.load(fp)

In [12]:
df_rows = []
wagner_nums = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
krs = [0.1, 1.0, 10.0]

for Wa_p in wagner_nums:
    for kr in krs:
        sim_metafile = get_simulation_metafile_path(resolution=1.0, kr=kr, Wa_p=Wa_p, gamma=15)
        try:
            row_data = read_simulation_metafile(sim_metafile)
        except FileNotFoundError:
            print(sim_metafile)
            continue
        df_rows.append(row_data)

df = pd.DataFrame(df_rows, dtype=float)

output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.001-0.1/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.001-1.0/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.001-10.0/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.01-0.1/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.01-1.0/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.01-10.0/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.1-0.1/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.1-1.0/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-0.1-10.0/15/simulation.json
output/reaction_distribution/40-40-75/unrefined/1.0/butler_volmer/0.001-1.0-0.1/15/simulation.jso

In [13]:
# df.head()
df.columns

RangeIndex(start=0, stop=0, step=1)

In [14]:
utils.make_dir_if_missing("figures/secondary_current/")

### $\frac{\kappa}{\sigma}$ = 0.01

In [15]:
df_001 = df[np.isclose(df["ionic to electronic conductivity ratio (kr)"], 0.1)]
slope_001 = []
x = df_001['Positive Wagner Number'].to_numpy()
y = df_001['Total resistance [Ω.cm2]'].to_numpy()
for i in range(len(wagner_nums) - 1):
    slope_001.append((y[i+1] - y[i])/(x[i+1] - x[i]))
fig, ax = plt.subplots(figsize=(5, 4.5))
ax.plot(df_001['Positive Wagner Number'], df_001['Total resistance [Ω.cm2]'], 'x-')
ax.set_box_aspect(1)
ax.semilogx()
ax.set_ylabel(r'R [$\Omega\mathrm{cm}^2$]', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\mathrm{Positive\ Electrode\ Wa}$', fontdict={'fontsize': 'xx-large'})
ax.set_title(r'$\frac{\kappa}{\sigma}$ = 0.1', fontdict={'fontsize': 'xx-large'})
plt.tight_layout()
plt.savefig(f'figures/secondary_current/resistance-kr0.1.png', bbox_inches="tight", dpi=1200)

KeyError: 'ionic to electronic conductivity ratio (kr)'

### $\frac{\kappa}{\sigma}$ = 1

In [ ]:
df_1 = df[np.isclose(df["ionic to electronic conductivity ratio (kr)"], 1)]
fig, ax = plt.subplots(figsize=(5, 4.5))
slope_1 = []
x = df_1['Positive Wagner Number'].to_numpy()
y = df_1['Total resistance [Ω.cm2]'].to_numpy()
for i in range(len(wagner_nums) - 1):
    slope_1.append((y[i+1] - y[i])/(x[i+1] - x[i]))
ax.plot(df_1['Positive Wagner Number'], df_1['Total resistance [Ω.cm2]'], 'x-')
ax.set_box_aspect(1)
ax.set_ylabel(r'R [$\Omega\mathrm{cm}^2$]', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\mathrm{Positive\ Electrode\ Wa}$', fontdict={'fontsize': 'xx-large'})
ax.semilogx()
ax.set_title(r'$\frac{\kappa}{\sigma}$ = 1', fontdict={'fontsize': 'xx-large'})
plt.tight_layout()
plt.savefig(f'figures/secondary_current/resistance-kr1.png', bbox_inches="tight", dpi=1200)

### $\frac{\kappa}{\sigma}$ = 100

In [ ]:
df_100 = df[np.isclose(df["ionic to electronic conductivity ratio (kr)"], 10)]
fig, ax = plt.subplots(figsize=(5, 4.5))
slope_100 = []
x = df_100['Positive Wagner Number'].to_numpy()
y = df_100['Total resistance [Ω.cm2]'].to_numpy()
for i in range(len(wagner_nums) - 1):
    slope_100.append((y[i+1] - y[i])/(x[i+1] - x[i]))
ax.plot(df_100['Positive Wagner Number'], df_100['Total resistance [Ω.cm2]'], 'x-')
ax.set_box_aspect(1)
ax.set_ylabel(r'R [$\Omega\mathrm{cm}^2$]', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\mathrm{Positive\ Electrode\ Wa}$', fontdict={'fontsize': 'xx-large'})
ax.semilogx()
ax.set_title(r'$\frac{\kappa}{\sigma}$ = 10', fontdict={'fontsize': 'xx-large'})
plt.tight_layout()
plt.savefig(f'figures/secondary_current/resistance-kr10.png', bbox_inches="tight", dpi=1200)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4.5))
ax.plot(df_001['Positive Wagner Number'], df_001['stdev i positive charge transfer (normalized)'], 'x-', label=r'$\frac{\kappa}{\sigma}$ = 0.1')
ax.plot(df_1['Positive Wagner Number'], df_1['stdev i positive charge transfer (normalized)'], 'x-', label=r'$\frac{\kappa}{\sigma}$ = 1')
ax.plot(df_100['Positive Wagner Number'], df_100['stdev i positive charge transfer (normalized)'], 'x-', label=r'$\frac{\kappa}{\sigma}$ = 10')
ax.legend(prop={'size': 11})
ax.semilogx()
ax.set_box_aspect(1)
ax.set_ylabel(r'$\frac{\sigma_{\mathrm{i}}}{i_{\mathrm{superficial}}}$', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\mathrm{Positive\ Electrode\ Wa}$', fontdict={'fontsize': 'xx-large'})
plt.tight_layout()
plt.savefig(f'figures/secondary_current/i-stdev-normalized.png', bbox_inches="tight", dpi=1200)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4.5))
ax.plot(df_001['Positive Wagner Number'], df_001['Current at insulated boundary [A]']/df_001['Current at right boundary [A]'], 'x-', label=r'$\frac{\kappa}{\sigma}$ = 0.1')
ax.plot(df_1['Positive Wagner Number'], df_1['Current at insulated boundary [A]']/df_1['Current at right boundary [A]'], 'x-', label=r'$\frac{\kappa}{\sigma}$ = 1')
ax.plot(df_100['Positive Wagner Number'], df_100['Current at insulated boundary [A]']/df_100['Current at right boundary [A]'], 'x-', label=r'$\frac{\kappa}{\sigma}$ = 10')

ax.semilogx()
ax.semilogy()
ax.set_ylim([1e-5, 1e-1])
ax.set_box_aspect(1)
ax.axhline(y=0.01, linestyle='--', label='1% mark')
ax.legend(prop={'size': 11}, bbox_to_anchor=(1.1, 1.05))
ax.set_ylabel(r'$\frac{I_{\mathrm{insulated}}}{I_{\mathrm{positive\ electrode}}}$', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\mathrm{Positive\ Electrode\ Wa}$', fontdict={'fontsize': 'xx-large'})
plt.tight_layout()
plt.savefig(f'figures/secondary_current/convergence.png', bbox_inches="tight", dpi=1200)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4.5))
ax.plot(x[1:], slope_001, label=r'K$_r$ = 0.1')
ax.plot(x[1:], slope_1, label=r'K$_r$ = 1')
ax.plot(x[1:], slope_100, label=r'K$_r$ = 10')
ax.semilogx()
ax.legend(prop={'size': 11})
ax.set_ylim([0, 3])
ax.set_xlim([1, 1e3]);
ax.set_ylabel(r'$\frac{\mathrm{d}R_{\mathrm{total}}}{\mathrm{d}\mathrm{Wa}}$', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\mathrm{Positive\ Electrode\ Wa}$', fontdict={'fontsize': 'xx-large'})
ax.set_box_aspect(1);
plt.tight_layout();
plt.savefig(f'figures/secondary_current/kinetics-limited.png', bbox_inches="tight", dpi=1200)

In [ ]:
# viskex.dolfinx.plot_mesh(domain, show_edges=False, opacity=0.5)

In [ ]:
# viskex.dolfinx.plot_mesh_tags(domain, ct, show_edges=False, opacity=0.5)

In [ ]:
# viskex.dolfinx.plot_mesh_tags(domain, ft, "facets", viskex.utils.values_in([12, 8]), show_edges=False, opacity=1)